# Locating longitudinal-transverse correlation in transverse phase space 

The goal of this notebook is to identify the "width" of the measured longitudinal-transverse correlation in the phase space distribution of the BTF bunch at the first emittance station. We will look at the energy distribution of particles within a boundary in 4D transverse phase space.

In [ ]:
import sys
import os
from os.path import join
import importlib
import numpy as np
import h5py
from tqdm.notebook import tqdm
from tqdm.notebook import trange
from plotly import graph_objects as go
from matplotlib import pyplot as plt
from ipywidgets import interactive
from ipywidgets import widgets
import proplot as pplt

sys.path.append('../../')
from tools import plotting as mplt
from tools import utils
from tools.utils import project

In [ ]:
pplt.rc['grid'] = False
pplt.rc['cmap.discrete'] = False
pplt.rc['cmap.sequential'] = 'viridis'
pplt.rc['figure.facecolor'] = 'white'

## Load data 

In [ ]:
folder = '_saved/2021-12-03-VS06/'

In [ ]:
info = utils.load_pickle(join(folder, 'info.pkl'))
info

In [ ]:
filename = info['filename']
coords = utils.load_stacked_arrays(join(folder, f'coords_{filename}.npz'))
shape = tuple([len(c) for c in coords])
print('shape:', shape)

In [ ]:
f = np.memmap(join(folder, f'f_{filename}.mmp'), shape=shape, dtype='float', mode='r')

Crop the 5D array.

In [ ]:
dims = ["x", "xp", "y", "yp", "w"]
units = ["mm", "mrad", "mm", "mrad", "MeV"]
dims_units = [f'{d} [{u}]' for d, u in zip(dims, units)]
prof_kws = dict(lw=0.5, alpha=0.7, color='white', scale=0.12)

In [ ]:
crop = (
    (0, f.shape[0]),
    (7, f.shape[1] - 7),
    (0, f.shape[2]),
    (10, f.shape[3] - 10),
    (15, f.shape[4] - 15),
)

fig, axes = pplt.subplots(ncols=5, figwidth=7, spanx=False, figheight=None)
for i, ax in enumerate(axes):
    p = utils.project(f, i)
    p = p / np.sum(p)
    ax.plot(p, color='black')
    ax.format(xlabel=dims[i] + ' [pixel]', xlim=(crop[i][0], crop[i][1] - 1))
# axes.format(yscale='log')
plt.show()

In [ ]:
ind = tuple([slice(c[0], c[1]) for c in crop])
f = f[ind]
coords = [c[ind[i]] for i, c in enumerate(coords)]

Clip small negative values.

In [ ]:
f_max = np.max(f)
f_min = np.min(f)
if f_min < 0.0:
    print(f'min(f) = {f_min}')
    print('Clipping to zero.')
    f = np.clip(f, 0.0, None)

## Rectangular slice

### 2D projections 

First view the 2D projections at integer slices. (Set `frac_thresh` to -3.0.)

In [ ]:
mplt.interactive_proj2d(f / f_max, coords=coords, dims=dims, units=units)

And at range slices.

In [ ]:
mplt.interactive_proj2d(f / f_max, coords=coords, dims=dims, units=units,
                        slice_type='range')

In [ ]:
frac_thresh = 10.0**-3.0

And all 2D projections.

In [ ]:
axes = mplt.corner(
    f,
    coords=coords,
    diag_kind='None',
    labels=dims_units,
    # norm='log',
    handle_log='floor',
    frac_thresh=frac_thresh,
    prof='edges',
    prof_kws=dict(lw=0.5, alpha=0.7, color='white', scale=0.12),
)
# plt.savefig('_output/corner_VS06.png')

### 1D projections 

In [ ]:
mplt.interactive_proj1d(f, coords=coords, dims=dims, units=units, default_ind=4,
                        slice_type='int')

In [ ]:
mplt.interactive_proj1d(f, coords=coords, dims=dims, units=units, default_ind=4,
                        slice_type='range')

In [ ]:
ind = [18, 18, 18, 30]
_dims = ["x", "x'", "y", "y'"]

fig, axes = pplt.subplots(ncols=4, figwidth=7, figheight=1.5)
axes.format(xlabel="w [MeV]", yticklabels=[], xlim=(-0.065, 0.065))
for i, ax in enumerate(axes):
    idx = utils.make_slice(4, np.arange(i + 1), ind[:i+1])
    pw = project(f[idx], 3 - i)
    pw = pw / np.sum(pw)
    ax.bar(coords[4], pw, color='black', width=1)
    ax.format(title=", ".join([r"${} \approx 0$".format(_dim) for _dim in _dims[:i+1]]))
axes.format(ylim=(0.0, axes[0].get_ylim()[1]), xspineloc='bottom', yspineloc='left', ylabel='Density')
plt.savefig('_output/w_slices.png')

## Contour slice

Select only those pixels above a threshold — within a contour — in 4D phase space. An example of contours in 2D phase space.

In [ ]:
f2d = project(f, (2, 3))
f2d = f2d / np.max(f2d)

fig, ax = pplt.subplots()
mplt.plot_image(
    f2d, x=coords[2], y=coords[3], ax=ax,
    contour=True, 
    contour_kws=dict(
        labels=True, 
        color='white',
        lw=0.4,
        alpha=1,
        levels=(0.001, 0.01, 0.1, 0.5, 1.0),
    ),
    colorbar=True,
)
ax.format(xlabel="y [mm]", ylabel="yp [mrad]")
# plt.savefig('contour_yyp.png')
plt.show()

Project the distribution onto 4D transverse phase space.

In [ ]:
ftr = project(f, axis=(0, 1, 2, 3))
ftr = ftr / np.max(ftr)

View the distribution of pixel values in the array.

In [ ]:
fig, ax = pplt.subplots()
ax.hist(ftr.ravel(), bins=20, color='black')
ax.format(yscale='log', ylabel='Count', xlabel='4D pixel value')
plt.show()

### Energy distribution 

Plot the energy distribution of a shrinking 4D phase space volume. The volume is defined by a contour in 4D phase space.

In [ ]:
def energy_proj(f, level=0.5, ftr=None, normalize=True):
    if ftr is None:
        ftr = np.sum(f, axis=-1)
    pw = np.sum(f[np.where(ftr > level)], axis=0)
    if normalize:
        pw = pw / np.sum(pw)
    return pw

In [ ]:
def update(level):
    pw = energy_proj(f, level, ftr=ftr, normalize=True)
    fig, ax = pplt.subplots(figsize=(4, 1.5))
    ax.format(xlabel=dims_units[4])
    ax.bar(coords[4], pw, color='black')
    plt.show()
    
interactive(update, level=(0.0, 0.99, 0.01))

Uncomment cells below for various plots of the same data.

In [ ]:
n = 20
levels = np.linspace(0.99, 0.01, n)
pws = np.array([energy_proj(f, level, ftr=ftr) for level in levels]) 
pws = pws / np.max(pws)

In [ ]:
fig, ax = pplt.subplots(figsize=(4, 1.75))
ax.pcolormesh(coords[4], levels[::-1], pws[::-1],
              colorbar=True, colorbar_kw=dict(label='Density (arb. units)', width=0.1))
ax.format(xlabel='w [MeV]', ylabel='4D contour level')
plt.show()

In [ ]:
cmap = pplt.Colormap('fire_r', left=0.0, right=0.9)
# cmap = pplt.Colormap('crest', left=0.0, right=1.0)

fig, ax = pplt.subplots(figsize=(4, 1.75))
ax.plot(pws[::-1].T, cycle=cmap, lw=1, colorbar=True, 
        colorbar_kw=dict(values=levels[::-1], label='4D contour level'))
plt.show()

In [ ]:
fig, ax = pplt.subplots(figsize=(4.0, 1.75))
for i, (color, pw) in enumerate(zip(pplt.get_colors('flare', len(pws)), pws)):
    x = coords[4]
    y = levels[i] + 0.075 * pw 
    ax.plotx(x, y, color='black')
ax.format(ylim=(-0.1, 0.1))
ax.format(ylabel='w [MeV]', xlabel="Fractional threshold in x-x'-y-y'")
plt.savefig('_output/4Dcontour_dE2.png')

In [ ]:
# Z = pws[::-1]
# X, Y = np.meshgrid(levels[::-1], coords[4], indexing='ij')    
# lines = []
# line_marker = dict(color='black', width=3)
# for x, y, z in zip(X, Y, Z):
#     lines.append(go.Scatter3d(x=x, y=y, z=z, mode='lines', line=line_marker))
# uaxis= dict(
#     gridcolor='rgb(255, 255, 255)',
#     zerolinecolor='rgb(255, 255, 255)',
#     showbackground=True,
#     backgroundcolor='rgb(230, 230,230)',
# )
# layout = go.Layout(
#     width=500,
#     height=500,
#     showlegend=False,
#     scene=dict(
#         xaxis=uaxis, 
#         yaxis=uaxis,
#         zaxis=uaxis,
#     ),
# )
# fig = go.Figure(data=lines, layout=layout)
# fig.show()

In [ ]:
# fig = go.Figure(data=[go.Surface(x=levels[::-1], y=coords[4], z=pws[::-1].T)])
# fig.update_layout(width=500, height=500)
# fig.show()

### Corner plot with slicing

We want to see what the distribution looks like as we slice it. First, plot the 2D projections of the 4D phase space distribution as the boundary is changed, along with the energy distribution of the pixels in the slice.

In [ ]:
def update(log=False, level=0.5):
    global ftr    
    axes = mplt.corner(
        np.ma.masked_where(ftr < level, ftr),
        coords=coords[:4],
        diag_kind='None',
        labels=dims_units,
        frac_thresh=frac_thresh,
        fill_value=0.0,
        norm='log' if log else None,
        handle_log='floor',
        prof='edges',
        prof_kws=dict(kind='step', lw=0.4, alpha=0.8),
    )
    pw = energy_proj(f, level, ftr=ftr, normalize=True)
    ax = axes[0, 2]
    for i in range(3):
        for j in range(i + 1):
            axes[i, j]._shared_x_axes.remove(ax)
            axes[i, j]._shared_y_axes.remove(ax)
    ax.axis('on')
    ax.plot(coords[4], pw, color='black')
    ax.format(
        xlim=(np.min(coords[4]), np.max(coords[4])),
        ylim=(0, 0.1),
        xspineloc='bottom', yspineloc='left',
        title='energy projection',
    )
    return axes
    
interactive(update, log=False, level=(0.0, 0.99, 0.01))

In [ ]:
# for i, level in enumerate(tqdm(np.linspace(0.0, 0.9, 25))):
#     axes = update(level=level)
#     axes.format(suptitle=f"threshold = {level:.2f}")
#     plt.savefig(f'_output/corner_slice_{i}.png', dpi=300)
#     plt.close()

Save a gif of this figure.

In [ ]:
def update(level):
    _ftr = np.ma.masked_where(ftr < level, ftr)
    _fxxp = project(_ftr, axis=(0, 1))
    _fyyp = project(_ftr, axis=(2, 3))
    plot_kws = dict(
        frac_thresh=frac_thresh,
        fill_value=0.0,
        norm=None,
        handle_log='floor',
        prof_kws=dict(kind='step', lw=0.4, alpha=0.8),
    )

    fig, axes = pplt.subplots(ncols=3, figwidth=6, share=False)
    for ax, (i, j) in zip(axes[:2], [(0, 1), (2, 3)]):
        mplt.plot_image(project(_ftr, (i, j)), x=coords[i], y=coords[j], ax=ax, 
                        # profx=True, profy=True, 
                        **plot_kws)
        ax.format(xlabel=dims_units[i], ylabel=dims_units[j])
    pw = energy_proj(f, level)
    ax = axes[2]
    ax.plot(energy_proj(f, level, ftr=ftr, normalize=True), color='black')
    ax.format(xlabel=dims_units[4], ylim=(0, 0.1), yticks=[])
    for ax in axes[:2]:
        ax.format(xlim=0.8*np.array(ax.get_xlim()), ylim=0.5*np.array(ax.get_ylim()))
        
interactive(update, level=(0.0, 0.99, 0.01))

Next, plot the 2D projections of the 5D distribution as the 4D slice boundary is changed. To slice the array, we compute a 4D mask in the transverse phase space, then copy the mask along the last axis of the array to get a 5D mask. *This works but is very slow...*

In [ ]:
def mask_4d(f, level=0.0, ftr=None):
    """Mask N-D array `f` based on contours of `f.sum(axis=-1)`."""
    if ftr is None:
        ftr = np.sum(f, axis=-1)
    condition = utils.copy_into_new_dim(ftr < level, f.shape[-1])
    return np.ma.masked_where(condition, f)

In [ ]:
def update(log, level):
    axes = mplt.corner(
        mask_4d(f, level, ftr=ftr),
        coords=coords,
        diag_kind='None',
        labels=dims_units,
        frac_thresh=frac_thresh,
        fill_value=0.0,
        norm='log' if log else None,
        handle_log='floor',
        prof='edges',
        prof_kws=dict(kind='step', lw=0.4, alpha=0.8),
    )
    plt.show()
    
interactive(update, log=False, level=(0.0, 0.99, 0.01))